# Monads, part 1: a design pattern

https://www.stephanboyer.com/post/9/monads-part-1-a-design-pattern


* Monad is a design pattern that allows us to redefine how function composition works.

* Before I define what a monad is, let’s develop some intuition with motivational examples.

## Example 1: exception handling (a.k.a. “Maybe”)

Let’s use Python to describe a function which computes the quotient of 100 and some divisor. 

In the special case where the divisor is zero, we’ll return None:

In [12]:
def divide100(divisor):
    if divisor == 0:
        return None
    else:
        return 100 / divisor

Let’s also write a function to compute the square root of a number.

If the input is negative, we’ll return `None`:

In [13]:
def sqrt(x):
    if x < 0:
        return None
    else:
        return x ** 0.5

Can we compose these functions?

What if we wanted to compute something like `sqrt(divide100(sqrt(x)))`? 
That would work as long as `x` is positive.

If we explicitly handle the case when any of those functions returns `None`, the code becomes much longer:

In [14]:
x = 1.0
a = sqrt(x)
if a is not None:
    b = divide100(a)
    if b is not None:
        c = sqrt(b)
c

10.0

You can imagine how tedious it would be if we had to do manual error checking like this for all of our function calls. 

Perhaps a better solution is to rewrite `divide100` and sqrt such that they each do the error handling themselves. 

For example, we might modify them as follows:

In [15]:
def composable_divide100(divisor):
    if divisor is None or divisor == 0:
        return None
    else:
        return 100 / divisor

def composable_sqrt(x):
    if x is None or x < 0:
        return None
    else:
        return x**0.5


Now we can evaluate expressions like `composable_sqrt(composable_divide100(composable_sqrt(x)))`.

When `x <= 0`, the entire expression evaluates to None just as we would expect.

Rather than modifying all of our functions to check for `None`, let’s write a wrapper function (let’s call it `bind`) to do the error handling for us. 

It takes a value (either a number or `None`) and a function (such as `divide100` or `sqrt`) and applies the function to that value. 

If the input is None, we skip the function application and just return `None`:

In [16]:
def bind(x, f):
    if x is None:
        return None
    else:
        return f(x)

Now we can compose these functions like: `bind(bind(bind(x, sqrt), divide100), sqrt)`. Sweet! 

We have a way to compose numerical functions that might fail.

You’ve essentially just implemented Haskell’s `Maybe` monad, which is a simple form of exception handling.

Let’s try some more complex examples.

## Example 2: vector operations (a.k.a. “List”)

We know that, mathematically, positive numbers have two square roots.
Let’s modify sqrt to return a list of values:

In [17]:
def sqrt(x):
    if x < 0:
        return []
    elif x == 0:
        return [0]
    else:
        return [x**0.5, -x**0.5]

So there are three cases we have to consider.

If `x` is positive, we return its two square roots.
If `x` is `0`, we return `[0]`.
If `x` is negative, we return the empty list.
Great!

Our sqrt function now makes more mathematical sense, at least for real numbers.

But we have the same problem as in Example 1—it is no longer composable with itself.

We can’t just compute `sqrt(sqrt(x))`, because the inner call to `sqrt` returns a `list`, and the outer call expects a number.

As before, we need to define a bind function to help us with composition:

In [18]:
def bind(x, f): 
    return [j for i in x for j in f(i)]

Here, bind takes a list of numbers x and a function f. 

The doubly-iterated list comprehension might look cryptic—you can think of it like this: We apply f to each value in x, which gives us a list of lists. 

Then we flatten the result into one list and return it. 

Now we can compute the square roots of a list of numbers, and then compute all the square roots of the results:

In [19]:
bind(bind([5, 0, 3], sqrt), sqrt) 

# [1.4953487812212205, -1.4953487812212205, 0, 1.3160740129524924, -1.3160740129524924]

[1.4953487812212205,
 -1.4953487812212205,
 0,
 1.3160740129524924,
 -1.3160740129524924]

It works!

But our original goal was to find the square roots of one number.

We could always write bind([x], sqrt) where x is a number, maybe it would be better to use a function to abstract the representation of our input.

Let’s call this function unit:

In [20]:
def unit(x):
    return [x]

Yep, it just puts a value into a list. 

It may seem unmotivated now, but we’ll write a more helpful unit function in the next example. 

Now we don’t have to put our input into a list—instead, we run it through unit, which puts it in the necessary representation:

In [21]:
bind(bind(unit(4), sqrt), sqrt) 

# [1.4142135623730951, -1.4142135623730951]

[1.4142135623730951, -1.4142135623730951]

Cool, we can now intelligently compose functions that might return several values! 

That’s basically Haskell’s List monad.

## Example 3: debug output (a.k.a. “Writer”)